In [8]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd 
from mypackage import cleaning_functions
from mypackage import scraping_functions as sf

In [9]:
game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
df = sf.scrape_game_data(game_url)

Chiefs vs. Lions 
 total plays: 132


In [19]:
df.head()

,Quarter,Time,Down,ToGo,Location,DET,KAN,Detail,EPB,EPA,field_side,yardline,play_start_time,Play_Type,possession,Yardage
1,1,15:00,,,KAN 35,0,0,"Harrison Butker kicks off 65 yards, touchback.",0.000,0.610,KAN,35,0.000000,Special Teams,DET,0.0
2,1,15:00,1,10,DET 25,0,0,David Montgomery up the middle for 7 yards (ta...,0.610,1.010,DET,25,0.000000,Run,DET,7.0
3,1,14:29,2,3,DET 32,0,0,Jared Goff pass incomplete short right,1.010,0.300,DET,32,0.516667,Pass,DET,0.0
4,1,14:25,3,3,DET 32,0,0,Jared Goff pass incomplete deep right intended...,0.300,-1.240,DET,32,0.583333,Pass,DET,0.0
5,1,14:19,4,3,DET 32,0,0,"Jack Fox punts 61 yards, returned by Richie Ja...",-1.240,0.320,DET,32,0.683333,Special Teams,DET,0.0


In [17]:
# seconds_left function
def seconds_left(plays):
    seconds_left = abs(plays['play_start_time'] - 60) * 60
    return seconds_left

In [26]:
def score_diff(plays):
    return plays.iloc[:,5].astype(float) - plays.iloc[:,6].astype(float)

In [36]:
# make adjusted score
#adjusted score = score / (seconds_left + 1) ^ gamma
def adjusted_score_calc(plays):
    gamma = .5

    adjusted_score = plays['score_diff'] / ((plays['seconds_left']) + 1) ** gamma

    return adjusted_score

In [34]:
df['score_diff'] = df.iloc[:,5].astype(float) - df.iloc[:,6].astype(float)

In [37]:
test_column = df.apply(adjusted_score_calc,axis =1)

In [13]:
df.tail()

,Quarter,Time,Down,ToGo,Location,DET,KAN,Detail,EPB,EPA,field_side,yardline,play_start_time,Play_Type,possession,Yardage
135,4,1:53,2,5,KAN 25,21,20,David Montgomery right guard for 3 yards (tack...,3.710,3.400,KAN,25,58.116667,Run,DET,3.0
137,4,1:47,3,2,KAN 22,21,20,David Montgomery up the middle for 2 yards (ta...,3.400,4.240,KAN,22,58.216667,Run,DET,2.0
139,4,1:42,1,10,KAN 20,21,20,Jared Goff kneels for -1 yards,4.240,3.560,KAN,20,58.300000,Run,DET,-1.0
140,4,1:05,2,11,KAN 21,21,20,Jared Goff kneels for -1 yards,3.560,2.740,KAN,21,58.916667,Run,DET,-1.0
141,4,0:39,3,12,KAN 22,21,20,Jared Goff kneels for -1 yards,2.740,1.930,KAN,22,59.350000,Run,DET,NaN


In [95]:
def win(plays):
    #plays = pd.DataFrame(plays)
    winning_team = None

    # Get the column names dynamically
    home_team = plays.columns[6]
    vis_team = plays.columns[5]

    Y = []

    # Compare who is the winning team
    if plays.iloc[-1, 5] >= plays.iloc[-1, 6]:
        winning_team = vis_team
    else:
        winning_team = home_team
        

    # Figure out who is winning at the end
    for _, play in plays.iterrows():
        if play['possession'] == play[winning_team]:
            Y.append(1)
        else:
            Y.append(0)
        
    
    return Y


'21'

In [99]:
test_column1 = df.apply(lambda row: win(pd.DataFrame(row).transpose()), axis=1)


Down The current down (1st, 2nd, 3rd, or 4th)
Score Difference in score between the two teams
Seconds Number of seconds remaining in the game
AdjustedScore Score/ 1 Seconds+
Spread Las Vegas pre-game point spread
TOTp Total points scored
Yardline Yards from own goal line
YTG Yards to go for a first down

In [101]:
df['seconds_left'] = df.apply(seconds_left,axis =1)
df['score_diff'] = df.iloc[:,5].astype(float) - df.iloc[:,6].astype(float)
df['adjusted_score'] = df.apply(adjusted_score_calc,axis=1)
df['win_team'] = df.apply(lambda row: win(pd.DataFrame(row).transpose()), axis=1)
# Define the column indices to drop
columns_to_drop = [5, 6]

# Drop the specified columns
new_data = df.drop(columns=df.columns[columns_to_drop])

# Optionally, you can also drop additional columns
additional_columns_to_drop = ['Quarter', 'Detail', 'Location', 'Time', 'win_team', 'play_start_time']
new_data = new_data.drop(columns=additional_columns_to_drop)

y = df['win_team']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_data, y, test_size=.2, random_state=42)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=400, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

#classification_report